# An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale

Reference:
* https://github.com/lucidrains/vit-pytorch
* https://github.com/huggingface/transformers/blob/v4.20.1/src/transformers/models/vit
* https://github.com/google-research/vision_transformer

## 1. Data

### 1.1. Data Augmentation and Regularization
* 공개된 코드에는 Aug는 `RandomResizedCrop`, `RandomHorizontalFlip`만 적용되어 있음. 
* 후속 논문에서 AugReg가 데이터 셋이 적은 경우에는 효과적이라고 함.
* 하지만, 데이터가 커질 수록 Aug, Reg가 큰 효과를 발휘하지 못함.
* 특히 Reg (e.g. dropout)의 경우는 오히려 성능을 하락시킴.
* 반면, Aug는 epoch을 더 늘리면 그래도 성능 향상에 도움이 됨.

## 2. Model

### 2.1. Input Representation 구현

<img src="assets/img_1.png">

### 2.2. Layer Norm의 위치.

https://aclanthology.org/P19-1176.pdf

<img src="assets/img_2.png">

### 2.3. einops로 구현하는 MHA

[einops](https://github.com/arogozhnikov/einops)
* einsum notation의 아이디어를 tensor manipulation으로 확장하여 사용할 수 있는 라이브러리
* 다양한 딥러닝 framework에 interoprational 하다.

<img src="assets/img_3.png">

## 3. Train

### 3.1. Experiment Details

<img src = "assets/img_4.png">

* ImageNet Pretraining 하는 경우에는 strong한 regulariztion 학습하는데 도움이 되었음.
* pretrain은 224 이미지에 대하여 진행함. fine-tuning은 384 이미지에 대하여 진행함.
    * finetuning을 다른 resolution에 대하여 진행하는 것을 common practice라 함.
* self-supervision MLM과 동일한 방식으로 해봤음.
    * 1) predicting only the mean, 3bit color (i.e., 1 prediction of 512 colors) ??
    * 2) predicting a 4 × 4 downsized version of the 16 × 16 patch with 3bit colors in parallel (i.e., 16 predictions of 512 colors) ??
    * 3) 3) regression on the full patch using L2 (i.e., 256 regressions on the 3 RGB channels).
    * 1) 옵션이 few-shot learning의 결과가 가장 잘 나왔고, 3)번은 결과가 잘 안나옴. 어찌되었든, image classification 기반의 pretrain보다는 성능이 안나옴.    

### 3.2. TRANSFORMER SHAPE

<img src = "assets/img_5.png">

* Figure 8 shows 5-shot performance on ImageNet for different configurations. 
* based on ViT model with 8 layers, D = 1024, DMLP = 2048 and a patch size of 32.
* We can see that scaling the depth results in the biggest improvements which are clearly visible up until 64 layer. (diminishing return after 16 layer.)
* witdth는 효과가 미미함.
* Decreasing the patch size and thus increasing the effective sequence length shows surprisingly robust improvements without introducing parameters.
    * patch_size 줄이면 seq_len 늘어남 상대적인 computation이 늘어남.
* 전반적으로 연산량이 늘어나면 비례하여 성능이 좋아짐. 

### 3.3. CLS vs GAP

<img src = "assets/img_6.png">


* CLS를 쓰거나, GAP를 쓰거나 성능에는 큰 차이가 없다. 다만 각각에 맞는 learning rate가 다르다.

### 3.4. Positional Embedding

<img src = "assets/img_7.png">


* No pos, 2-D Pos, Rel Pos 다해봤는데, 1-D가 전반적인 성능이 가장 잘 나왔다.

---

# 함께 보면 좋을 것들.

### [How to train your ViT? Data, Augmentation, and Regularization in Vision Transformers](https://arxiv.org/pdf/2106.10270.pdf)
* 다양한 Data, AugReg들에 대하여 ViT를 어떻게 학습하면 좋을지 5만개의 ViT모델을 학습하면서 얻은 findings를 공유하는 페이퍼.

### [Surrogate Gap Minimization Improves Sharpness-Aware Training](https://arxiv.org/pdf/2203.08065.pdf)
> Empirically, GSAM consistently improves generalization (e.g., +3.2% over SAM and +5.4% over AdamW on ImageNet top-1 accuracy for ViT-B/32)
* GSAM(Surrogate Gap Guided Sharpness-Aware Minimization) optimizer.
* 기존 AdamW보다 ImageNet top-1 accuracy가 약 5.4% 상승함. (ViT-B/32)
* ICLR2022 accepted paper

### 다양한 vision transformers 관련 페이퍼
